In [1]:
!pip install flask requests

In [2]:
%%writefile app.py
from flask import Flask, request, jsonify
import re

app = Flask(__name__)

# Pre-made responses
responses = {
    "hello": "Hi there! How can I help you today?",
    "bye": "Goodbye! Have a nice day.",
    "default": "I'm not sure how to respond to that."
}

# Simple function to check if the user is angry using rude word matching
def is_angry(message):
    # List of rude words (example list, not exhaustive)
    rude_words = ["stupid", "idiot", "dumb", "hate", "bastard"]
    message_lower = message.lower()
    for word in rude_words:
        if re.search(r"\b" + re.escape(word) + r"\b", message_lower):
            return True
    return False

@app.route('/chat', methods=['POST'])
def chat():
    data = request.get_json()
    user_message = data.get("message", "")

    if is_angry(user_message):
        reply = "I sense some anger. Let's try to keep the conversation calm."
    else:
        # Basic rule-based matching
        if "hello" in user_message.lower():
            reply = responses.get("hello")
        elif "bye" in user_message.lower():
            reply = responses.get("bye")
        else:
            reply = responses.get("default")

    return jsonify({"reply": reply})

if __name__ == '__main__':
    # The server will run on 0.0.0.0:5000
    app.run(host='0.0.0.0', port=5000)


Writing app.py


In [4]:
!nohup python app.py &


nohup: appending output to 'nohup.out'


In [5]:
import socket
import time

time.sleep(1)

def check_port(host, port):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        try:
            sock.connect((host, port))
            print(f"Port {port} on {host} is open!")
        except ConnectionRefusedError:
            print(f"Port {port} on {host} is closed.")

check_port("127.0.0.1", 5000)

Port 5000 on 127.0.0.1 is open!


In [6]:
import requests

url = "http://127.0.0.1:5000/chat"

# Example conversation messages
messages = [
    "Hello",                        # Should trigger the greeting response
    "You are such an idiot!",       # Contains rude word -> angry detection
    "Bye"                           # Should trigger the goodbye response
]

for msg in messages:
    payload = {"message": msg}
    response = requests.post(url, json=payload)
    if response.ok:
        print(f"User: {msg}")
        print(f"Chatbot: {response.json().get('reply')}\n")
    else:
        print("Failed to connect to the chatbot service.")


User: Hello
Chatbot: Hi there! How can I help you today?

User: You are such an idiot!
Chatbot: I sense some anger. Let's try to keep the conversation calm.

User: Bye
Chatbot: Goodbye! Have a nice day.

